In [34]:
# nbi:hide_in
import numpy as np
from bqplot import pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import nbinteract as nbi
from ipywidgets import *


layout={'width':'350px','margin':'0 0 0 0','padding':'0 0 0 0'}
style={'description_width':'150px'}
phs=FloatSlider(min=0,max=200,value=120,description='N\u2082 Pressure (bar)',layout=layout,style=style)
Dint=FloatSlider(min=0,max=500,value=284,description='\u2300 Intensification cyl.(mm)',layout=layout,style=style)
Dinj=FloatSlider(min=0,max=300,value=181,description='\u2300 Injection cyl(mm)',layout=layout,style=style)
Dpl=FloatSlider(min=0,max=200,value=120,description='\u2300 Piston (mm)',layout=layout,style=style)
Ag=FloatSlider(min=0,max=50,value=15.5,description='Gate area (cm\u00b2)',layout=layout,style=style)
vpl=FloatSlider(min=0,max=10,value=4.2,description='Plunger speed (m/s)',layout=layout,style=style)
Ti=FloatSlider(min=500,max=800,value=630,description='Metal temp at gate \u00b0C',layout=layout,style=style)
Tf=FloatSlider(min=500,max=800,value=570,description='Min metal temp to flow \u00b0C',layout=layout,style=style)
Td=FloatSlider(min=50,max=400,value=180,description='Die temp \u00b0C',layout=layout,style=style)
S=FloatSlider(min=0,max=100,value=20,description='Fraction solid (%)',layout=layout,style=style)
tw=FloatSlider(min=0,max=10,value=4,description='Avg wall thickness (mm)',layout=layout,style=style)
dl=FloatSlider(min=0,max=3,value=2.39,description='density of Al (g/cm\u00b3)',layout=layout,style=style)
Cd=FloatSlider(min=0,max=1,value=0.5,description='Co-effecient of discharge',layout=layout,style=style)
vgmax=FloatSlider(min=0,max=100,value=55,description='Max gate velocity (m/s)',layout=layout,style=style)
vgmin=FloatSlider(min=0,max=100,value=25,description='Min gate velocity (m/s)',layout=layout,style=style)
Wp=FloatSlider(min=0,max=30000,value=5040,description='Part weight (gms)',layout=layout,style=style)
Wof=FloatSlider(min=0,max=10000,value=688,description='Overflow weight (gms)',layout=layout,style=style)
Wrun=FloatSlider(min=0,max=20000,value=2920,description='Runner weight (gms)',layout=layout,style=style)
Aproj=FloatSlider(min=0,max=5000,value=1750,description='Projected area (cm\u00b2)',layout=layout,style=style)
Ls=FloatSlider(min=0,max=200,value=89.2,description='Sleeve length (cm)',layout=layout,style=style)
vdry=FloatSlider(min=0,max=10,value=6.5,description='Dry plunger speed (m/s)',layout=layout,style=style)
tb=FloatSlider(min=0,max=10,value=3.5,description='Biscuit thickness (cm)',layout=layout,style=style)
Tliq=IntSlider(min=550,max=600,value=582,description='Liquidus temp in \u00b0C',
              layout=layout,style=style)
Tsol=IntSlider(min=500,max=550,value=516,description='Solidus temperature in \u00b0C',
              layout=layout,style=style)
gt=Text(description='Part name',layout=layout,style=style)

def calc_1(phs,Dint,Dinj,Dpl,Ag,vpl,vdry,Ti,Tf,Td,S,tw,dl,Cd,tb,
           vgmax,vgmin,Wp,Wof,Wrun,Aproj,Ls,Tliq,Tsol,gt):
    Z=3.8
    g=9.81
    cp=962.5 # specific heat in J/kg deg K
    Lf=388442 # Latent heat of fusion in J/kg
    k=96.1 # Thermal conductivity in J/s m degK
    hsurf=65000 #Convective heat transfer coeffecient in W/m^2 degK
    if Ti>Tliq:
        Te=Tliq # Solidification analysis start temperature
    else :
        Te=Ti
    Aint=np.pi*(Dint/10)**2/4
    Ainj=np.pi*(Dinj/10)**2/4
    Finj=phs*Ainj*10/1000
    Apl=np.pi*(Dpl/10)**2/4
    Pm=Finj/Apl*1000/10
    Vm=(Wp+Wof)/dl
    tact=Vm/(Apl*vpl*100)
    Q=Vm/tact
    
    vg=Apl*vpl/Ag
    hc=k/2/(tw/1000)*(3.3+0.02*(2*vg*tw*dl*cp/k)**0.8) #convective heat transfer computed 
                                                      #  variable in W/m^2 degK
    h=(hc*hsurf)/(hc+hsurf) # Convective heat transfer coeffecient 
                            #computed variable in W/m^2 degK
    trec=tw/2*dl*cp/h*(-np.log((Tliq-Td-S/100*(Tliq-Tsol))/(Ti-Td))-
                      Lf/(cp*(Tliq-Tsol))*np.log((Tliq-Td-S/100*(Tliq-Tsol))/(Te-Td))) #Flow time
                                            #Elapsed time from injection to target fraction solid
    xfs=vg*trec
    Agt=Vm/1e6/xfs # Target gate area to achieve conditions
    Qrec=Vm/trec
    Pmax=Aint/Apl*phs
    Wshot=Wp+Wof+Wrun
    Rf=Wshot/dl/(Apl*Ls)
    Qmax=Apl*vdry*100
    Wb=tb*Apl*dl
    s1=(1-Rf)*Ls
    s2=(Wrun-Wb)/dl/Apl
    L1st=s1+s2
    Phase2pos=s1+s2
    Wb=tb*Apl*dl
    L2nd=Ls-(L1st+tb)
    vgmaxp=(dl/(2*g*100))*(vgmax*100/Cd)**2/1000/1.01972
    vgminp=(dl/(2*g*100))*(vgmin*100/Cd)**2/1000/1.01972
    vgp=(dl/(2*g*100))*(vg*100/Cd)**2/1000/1.01972
    Flreq=Aproj*Pmax*10/1000/g
    ax= plt.figure(figsize=(20,7))
    plt.clear()
    plt.title('PQ\u00B2 '+str(gt))
#     plt.subplots_adjust(left=0.1,top=0.93,right=1)
    plt.plot([0,Qmax**2],[Pm,0],'b-' ,labels=['Machine line'],display_legend=True)
    plt.plot([Q**2,Q**2],[vgp,vgp],'rs',labels=['Actual process point'],display_legend=True)
    plt.plot([0,Qmax**2*2],[0,vgp/Q**2*Qmax**2*2],'m-',labels=['Die line'],display_legend=True,ax=ax)
    plt.hline(vgmin,color=['c-'],labels=['Min gate velocity'],display_legend=True)
    plt.hline(vgmaxp,color=['r-'],labels=['Max gate velocity'],display_legend=True)
    plt.plot([Qrec**2]*2,[vgminp,vgmaxp],labels=['min flow required'],display_legend=True)
    plt.xlim(0,Qmax**2*1.2)
    plt.ylim(0,Pm*1.2)
    ax.legend_location='top-right'
    plt.xlabel('Flow (Q\u00B2) in (cm\u00B3/s)\u00B2')
    plt.ylabel('Metal Pressure in Sleeve (Pm) in bar')
#     ax.text(Qmax**2,vgmaxp+5,'vgmax = '+str(vgmax)+' m/s',ha='center')
#     ax.text(Qmax**2,vgminp+5,'vgmin = '+str(vgmin)+' m/s',ha='center')
#     ax.text(Q**2*1.05,vgp,'gate velocity: '+str(int(round(vg,0)))+' m/s',ha='left')
    
    plt.show()
    print('phase2pos: {} mm \nLocking force: {} tons \nIntensification pressure: {} bar \nFill ratio: {}\nShot weight: {} gms'.format(round(Phase2pos*10,0),round(Flreq,0),round(Pmax,0),round(Rf*100,0),Wshot))
    print('hc ={} W/m^2 degK (convective heat transfer computed)\
          \nh= {} W/m^2 degK (convective heat transfer computed)\
          \ntrec= {} s (Target flow time, Elapsed time from injection to target fraction solid)\
          \nxfs = {}  \
          \nAgt = {} cm^2 (Target gate area to achieve conditions specified)' \
         .format(round(hc,0),round(h,0),round(trec,4),xfs,round(Agt*1e4,2)))
    print('Pressure of metal in sleeve={} \
          \nFill time ={} \
          \nFlow (Qrec)= {} \
          \nFill time rec= {}' \
          .format(round(Pm,0),round(tact,4),round(Qrec,0),round(trec,4)))

display(widgets.HBox([VBox([phs,Dint,Dinj,Dpl,Ag,vpl,vdry,Tliq]),VBox([vgmax,vgmin,Wp,Wof,Wrun,Ti,Tf,Td]), 
                      VBox([S,tw,dl,Cd,tb,Tsol,Aproj,Ls,gt])]),
        widgets.interactive_output(calc_1,{'phs':phs,'Dint':Dint,'Dinj':Dinj,'Dpl':Dpl,
                                           'Ag':Ag,'vpl':vpl,'vdry':vdry,'Ti':Ti,
                                           'Tf':Tf,'Td':Td,'S':S,'tw':tw,'dl':dl,'Cd':Cd,
                                           'tb':tb,'vgmax':vgmax,'vgmin':vgmin,'Wp':Wp,
                                           'Wof':Wof,'Wrun':Wrun,'Aproj':Aproj,'Ls':Ls,
                                           'Tliq':Tliq,'Tsol':Tsol,'gt':gt}))

Output()

In [36]:
nbi.publish('sprabhakar2006/PQ2/master','PQ2.1.ipynb')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving notebook... Saved 'PQ2.1.ipynb'.
Converting notebook...



Successfully converted!

<a href="PQ2.1.html" target="_blank" download>Click to download your webpage.</a>

To host your webpage, see the documentation:
<a href="https://www.nbinteract.com/tutorial/tutorial_publishing.html"
        target="_blank">
    https://www.nbinteract.com/tutorial/tutorial_publishing.html
</a>
